In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import broadcast

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = f"\
--conf spark.hadoop.fs.s3a.endpoint=http://minio-ml-workshop:9000 \
--conf spark.hadoop.fs.s3a.access.key=minio \
--conf spark.hadoop.fs.s3a.secret.key=minio123 \
--conf spark.hadoop.fs.s3a.path.style.access=true \
--conf spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem \
--conf spark.hadoop.fs.s3a.multipart.size=104857600 \
--packages org.apache.hadoop:hadoop-aws:3.2.0,org.postgresql:postgresql:42.3.3 \
--master spark://{os.environ['SPARK_CLUSTER']}:7077 pyspark-shell "

# Create the spark application
spark = SparkSession \
    .builder \
    .appName("Enrich flights data") \
    .getOrCreate()

In [3]:
keys = ["year", "month", "day", "flight_number"]

df_flights = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://pg-flights-data:5432/postgres") \
    .option("dbtable", "flights") \
    .option("user", "postgres") \
    .option("password", "postgres") \
    .option("driver", "org.postgresql.Driver") \
    .option("numPartitions", 5000) \
    .option("partitionColumn", "flight_number") \
    .option("lowerBound", 1)\
    .option("upperBound", 10000)\
    .load()

print(f"Partition count:{df_flights.rdd.getNumPartitions()}")

df_airlines = spark.read\
                .options(delimeter=',', inferSchema='True', header='True') \
                .csv("s3a://airport-data/airlines.csv")
df_airports = spark.read\
                .options(delimiter=',', inferSchema='True', header='True') \
                .csv("s3a://airport-data/airports.csv")

#df_flights.printSchema()
#df_airlines.printSchema()
#df_airports.printSchema()

Partition count:5000


In [4]:
from pyspark.sql.functions import col

df_airlines = df_airlines.select([col(c).alias("AL_"+c) for c in df_airlines.columns])
df_o_airports = df_airports.select([col(c).alias("ORIG_"+c) for c in df_airports.columns])
df_d_airports = df_airports.select([col(c).alias("DEST_"+c) for c in df_airports.columns])
#df_airlines.printSchema()
#df_o_airports.printSchema()
#df_d_airports.printSchema()

In [5]:
df_flights = df_flights\
    .join(broadcast(df_airlines), df_flights.airline == df_airlines.AL_IATA_CODE)\
    .join(broadcast(df_o_airports), df_flights.origin_airport == df_o_airports.ORIG_IATA_CODE)\
    .join(broadcast(df_d_airports), df_flights.destination_airport == df_d_airports.DEST_IATA_CODE)

#df_flights.printSchema()

In [6]:
df_flights.rdd.getNumPartitions()

5000

In [ ]:
output_location = "s3a://flights-data/flights"

df_flights.write.mode("overwrite")\
    .option("header","true")\
    .format("parquet").save(output_location)

In [11]:
spark.stop()